<p style="align: center;"><img align=center src="https://mailfit.com/wp-content/uploads/2019/11/lego-5.png"  width=900></p>
<h1 style="text-align: center;"><b>«Домашняя работа» - Ускорение тестирования</b></h3>


## Импорт библиотек

In [5]:
import numpy as np
import pandas as pd

import scipy

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

## Импорт данных

In [6]:
data_path = "https://raw.githubusercontent.com/a-milenkin/Datasetes_for_Piplines/main/SkillBox/ab_speedup.csv"
data = pd.read_csv(data_path, index_col=0)
print("Размеры датасета", data.shape)
data.head(6)

Размеры датасета (50000, 2)


,group,metrica
strata,,
11,history,42.631346
6,history,14.844453
4,history,2.362768
37,history,79.494017
18,history,-22.627788
34,history,28.679378


<div class="alert alert-info">
<b>Про Датасет</b>
    
Датасет представляет собой результаты некоторого А/B-Теста. 

* `strata` - Некоторая информация по страту (группа), к которой относится пользователь. 
* `group` - Группа эксперимента - тестовая, контрольная или прошлая история. 
* `metrica` - Целивая метрика
    
</div>

In [10]:
data = data.reset_index()  # Сброс индекса
print(data.head())  # Проверьте результат

   strata    group    metrica
0      11  history  42.631346
1       6  history  14.844453
2       4  history   2.362768
3      37  history  79.494017
4      18  history -22.627788


In [11]:
# Проверим уникальные значения в группах и стратах
print(data['group'].value_counts())
print(data['strata'].value_counts())

# Посмотрим общую статистику
data.describe()

group
B          21430
A          21427
history     7143
Name: count, dtype: int64
strata
11    1000
6     1000
4     1000
37    1000
18    1000
34    1000
9     1000
16    1000
43    1000
15    1000
20    1000
25    1000
0     1000
45    1000
23    1000
35    1000
21    1000
46    1000
22    1000
17    1000
3     1000
41    1000
44    1000
12    1000
13    1000
24    1000
1     1000
26    1000
7     1000
30    1000
27    1000
2     1000
8     1000
40    1000
33    1000
10    1000
36    1000
31    1000
32    1000
39    1000
19    1000
5     1000
28    1000
29    1000
38    1000
47    1000
42    1000
14    1000
49    1000
48    1000
Name: count, dtype: int64


,strata,metrica
count,50000.000000,50000.000000
mean,24.500000,35.654712
std,14.431014,34.291229
min,0.000000,-147.125702
25%,12.000000,13.154502
50%,24.500000,28.380041
75%,37.000000,53.585312
max,49.000000,230.607984


<div class="alert alert-info">

<b>Задание:</b>    
    
Найти различие между группами, увеличив чувствительность тестов тремя способами поочередно:
* Бутстрап
* Стратификация
* CUPED
    
Вспомогательная статья Валерия Бабушкина про стратификацию и CUPED:

* habr.com/ru/company/yandex/blog/497804/

## Метод 1: Бутстрап
Бутстрап позволяет проводить сравнение метрик между тестовой и контрольной группами, увеличивая чувствительность анализа.

Шаги:

1. Разделим данные на тестовую и контрольную группы.
2. Проведем бутстрап для оценивания различий в метрике.

In [ ]:
# Разделение данных на группы
control = data[data['group']== 'A']['metrica']
test = data[data['group'] == 'B']['metrica']

# Bootstrap function
def bootstrap(data1, data2, n_bootstrap=1000):
    boot_means_diff = []
    for _ in range(n_bootstrap):
        sample1 = np.random.choice(data1, size=len(data1), replace=True)
        sample2 = np.random.choice(data2, size=len(data2), replace=True)
        boot_means_diff.append(np.mean(sample2)-np.mean(sample1))
    return np.percentile(boot_means_diff, [2.5, 97.5])

# Bootstrap for difference
conf_int = bootstrap(control, test)
print(f"95% доверительный интервал для разницы: {conf_int}")

95% доверительный интервал для разницы: [-1.45780095 -0.17662632]


###  Метод 2: Стратификация
Стратификация использует информацию о стратах для повышения точности оценок.

Шаги:

1. Группировка данных по `strata` и расчет средних значений метрик внутри каждой группы.
2. Проведение сравнения между тестовой и контрольной группами с учетом страт.

In [15]:
# Mean value metrics by strata
starata_mean = data.groupby(['strata', 'group'])['metrica'].mean().unstack()

# Сраsвнение метрики с учетом страт
strata_diff = starata_mean['A'] - starata_mean['B']
overall_diff = strata_diff.mean()

print(f"Средняя разница метрик между группами с учетом страт: {overall_diff}")


Средняя разница метрик между группами с учетом страт: 0.5604612024686025


### Метод 3: CUPED
CUPED (Controlled Pre-Experiment Data) использует предтестовые данные для уменьшения дисперсии целевой метрики.

Шаги:

1. Добавим предположение о предтестовых данных (например, создадим синтетический столбец `pre_metrica`).
2. Проведем расчет CUPED-метрики и сравним группы.

In [17]:
# Создаем синтетические предпредставительные данн
np.random.seed(42)
data['pre_metrica'] = data['metrica'] + np.random.normal(0, 0.1, size=len(data))

# Рассчитываем CUPED
theta = np.corrcoef(data['pre_metrica'], data['metrica'])[0,1]
data['cuped_metrica'] = data['metrica'] - theta * (data['pre_metrica'] - np.mean(data['pre_metrica']))

# Сравнение групп по CUPED
cuped_B = data[data['group'] == 'B']['cuped_metrica']
cuped_A = data[data['group'] == 'A']['cuped_metrica']

# Расчет среднего значения и доверительного интервала
mean_diff_cuped = np.mean(cuped_A) - np.mean(cuped_B)
conf_int_cuped = bootstrap(cuped_B, cuped_A)

print(f"Средняя разница по CUPED: {mean_diff_cuped}")
print(f"95% доверительный интервал для разницы (CUPED): {conf_int_cuped}")

Средняя разница по CUPED: -0.00044638494333781864
95% доверительный интервал для разницы (CUPED): [-0.00235242  0.00150289]
